In [1]:
import emat
emat.__version__, emat.__path__

('0.1.4', ['/Users/jpn/Git/private-emat/emat'])

In [2]:
import ema_workbench
ema_workbench.__version__, ema_workbench.__path__

('2.1.CS1', ['/Users/jpn/Git/EMAworkbench/ema_workbench'])

In [3]:
import os, numpy, pandas, functools
from emat.util.xmle import Show
from emat.viz.scatter import scatter_graph

In [4]:
from sklearn.ensemble import VotingRegressor
from sklearn.multioutput import RegressorChain
from sklearn.gaussian_process import GaussianProcessRegressor
from emat.multitarget import DetrendedMultipleTargetRegression

In [5]:
road_test_scope_file = emat.package_file('model','tests','road_test.yaml')

s = emat.Scope(road_test_scope_file)
s

<emat.Scope with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [6]:
s.info()

name: EMAT Road Test
desc: prototype run
constants:
  free_flow_time = 60
  initial_capacity = 100
uncertainties:
  alpha = 0.1 to 0.2
  beta = 3.5 to 5.5
  input_flow = 80 to 150
  value_of_time = 0.001 to 0.08
  unit_cost_expansion = 95.0 to 145.0
  interest_rate = 0.025 to 0.04
  yield_curve = -0.0025 to 0.02
levers:
  expand_capacity = 0.0 to 100.0
  amortization_period = 15 to 50
  debt_type = categorical
  interest_rate_lock = boolean
measures:
  no_build_travel_time
  build_travel_time
  time_savings
  value_of_time_savings
  net_benefits
  cost_of_capacity_expansion
  present_cost_expansion


In [7]:
db = emat.SQLiteDB()

In [8]:
s.store_scope(db)

In [9]:
from emat.experiment.experimental_design import design_experiments
design = design_experiments(s, db=db, n_samples_per_factor=10, sampler='lhs')
design.head()


,alpha,amortization_period,beta,debt_type,expand_capacity,input_flow,interest_rate,interest_rate_lock,unit_cost_expansion,value_of_time,yield_curve,free_flow_time,initial_capacity
experiment,,,,,,,,,,,,,
1,0.184682,38,5.237143,Rev Bond,18.224793,115,0.031645,False,118.213466,0.022518,0.015659,60,100
2,0.166133,36,4.121963,Paygo,87.525790,129,0.037612,True,141.322696,0.052306,0.007307,60,100
3,0.198937,44,4.719838,GO Bond,45.698048,105,0.028445,False,97.783320,0.011923,-0.001545,60,100
4,0.158758,42,4.915816,GO Bond,51.297546,113,0.036234,True,127.224901,0.077527,0.004342,60,100
5,0.157671,42,3.845952,Paygo,22.824149,133,0.039257,False,107.820482,0.027270,0.001558,60,100


In [10]:
from emat.model.core_python import PythonCoreModel
from emat.model.core_python import Road_Capacity_Investment


In [11]:
cm = PythonCoreModel(Road_Capacity_Investment, scope=s, db=db)

In [12]:
# from ema_workbench import SequentialEvaluator

In [13]:
lhs_results = cm.run_experiments(design=design)
lhs_results.head()

,alpha,beta,input_flow,value_of_time,unit_cost_expansion,interest_rate,yield_curve,expand_capacity,amortization_period,debt_type,interest_rate_lock,no_build_travel_time,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion,present_cost_expansion
experiment,,,,,,,,,,,,,,,,,,
1,0.184682,5.237143,115,0.022518,118.213466,0.031645,0.015659,18.224793,38,Rev Bond,False,83.038716,69.586789,13.451927,34.835362,-79.515515,114.350877,2154.415985
2,0.166133,4.121963,129,0.052306,141.322696,0.037612,0.007307,87.525790,36,Paygo,True,88.474313,62.132583,26.341730,177.741192,-205.321480,383.062672,12369.380535
3,0.198937,4.719838,105,0.011923,97.783320,0.028445,-0.001545,45.698048,44,GO Bond,False,75.027180,62.543328,12.483852,15.629037,-151.944318,167.573355,4468.506839
4,0.158758,4.915816,113,0.077527,127.224901,0.036234,0.004342,51.297546,42,GO Bond,True,77.370428,62.268768,15.101660,132.298475,-167.624871,299.923347,6526.325171
5,0.157671,3.845952,133,0.027270,107.820482,0.039257,0.001558,22.824149,42,Paygo,False,88.328990,72.848428,15.480561,56.146908,-3.972940,60.119848,2460.910705


In [14]:
# reload_results = m.read_experiments(design_name='lhs')
# reload_results.head()


In [15]:
mm = cm.create_metamodel_from_design('lhs')
mm

<emat.PythonCoreModel "EMATMeta", metamodel_id=1 with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [16]:
from emat.multitarget.stacked import DetrendedStackedSingleTargetRegression

mm_sst = cm.create_metamodel_from_design(
    'lhs',
    regressor=DetrendedStackedSingleTargetRegression
)
mm_sst

<emat.PythonCoreModel "EMATMeta", metamodel_id=2 with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [17]:
import emat.multitarget.boosting
import imp
imp.reload(emat.multitarget.boosting)
from emat.multitarget.boosting import BoostedRegressor

In [18]:
from emat.multitarget.linear import LinearRegression
#from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, RationalQuadratic as RQ
from emat.multitarget.anisotropic import AnisotropicGaussianProcessRegressor as AGPR
from emat.multitarget.base import MultiOutputRegressor
from emat.multitarget import MultipleTargetRegression
from emat.multitarget.ensemble import EnsembleRegressorChain2
from emat.multitarget.ensemble import DetrendedEnsembleRegressorChains
from emat.multitarget.chained import RegressorChain


In [19]:
boosts = BoostedRegressor([
    LinearRegression(
        frame_out=True,   
    ),
    MultipleTargetRegression(
    ),
])

In [20]:
mm_boo = cm.create_metamodel_from_design('lhs', regressor=boosts,)
mm_boo

<emat.PythonCoreModel "EMATMeta", metamodel_id=3 with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [21]:
mm_lr = cm.create_metamodel_from_design('lhs', regressor=LinearRegression(frame_out=True))
mm_lr

<emat.PythonCoreModel "EMATMeta", metamodel_id=4 with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [22]:
boost1 = BoostedRegressor([
    LinearRegression(
        frame_out=True,   
    ),
    AGPR(
        n_restarts_optimizer=250,
    ),
])

In [23]:
mm_rc = cm.create_metamodel_from_design('lhs', regressor=RegressorChain(boost1))
mm_rc

<emat.PythonCoreModel "EMATMeta", metamodel_id=5 with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [24]:
mm_lrc = cm.create_metamodel_from_design('lhs', regressor=LinearRegression(frame_out=True))
mm_lrc

<emat.PythonCoreModel "EMATMeta", metamodel_id=6 with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [25]:
mm_erc1 = cm.create_metamodel_from_design('lhs', regressor=EnsembleRegressorChain2(n_chains=1))
mm_erc1

<emat.PythonCoreModel "EMATMeta", metamodel_id=7 with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [26]:
mm_erc = cm.create_metamodel_from_design('lhs', regressor=EnsembleRegressorChain2(n_chains=6))
mm_erc

<emat.PythonCoreModel "EMATMeta", metamodel_id=8 with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [27]:
mm_erc5 = cm.create_metamodel_from_design('lhs', regressor=EnsembleRegressorChain2(n_chains=2, cv=5))
mm_erc5

<emat.PythonCoreModel "EMATMeta", metamodel_id=9 with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [28]:
validation_data = design_experiments(s, db=db, n_samples_per_factor=10, sampler='lhs', random_seed=444)

In [29]:
validation_data.head()

,alpha,amortization_period,beta,debt_type,expand_capacity,input_flow,interest_rate,interest_rate_lock,unit_cost_expansion,value_of_time,yield_curve,free_flow_time,initial_capacity
experiment,,,,,,,,,,,,,
111,0.118865,27,4.595404,GO Bond,41.407414,141,0.031256,True,127.842399,0.026021,-0.002180,60,100
112,0.144928,27,4.058454,Paygo,69.520526,114,0.034399,False,128.643815,0.037977,0.017485,60,100
113,0.140603,21,4.660208,GO Bond,3.283425,114,0.039105,True,130.959208,0.074024,-0.001967,60,100
114,0.115624,42,4.754447,Rev Bond,86.519772,145,0.038980,False,143.229074,0.071638,0.007511,60,100
115,0.150072,17,5.126105,Paygo,67.684449,144,0.032431,False,126.025730,0.021973,-0.001872,60,100


In [30]:
val = {}

In [31]:
val['cm'] = cm.run_experiments(design=validation_data)

In [32]:
val['mm'] = mm.run_experiments(design=validation_data)

In [33]:
val['rc'] = mm_rc.run_experiments(design=validation_data)

In [34]:
val['boo'] = mm_boo.run_experiments(design=validation_data)

In [35]:
val['sst'] = mm_sst.run_experiments(design=validation_data)

In [36]:
val['lr'] = mm_lr.run_experiments(design=validation_data)

In [37]:
val['erc1'] = mm_erc1.run_experiments(design=validation_data)

In [38]:
val['erc'] = mm_erc.run_experiments(design=validation_data)

In [49]:
val['erc5'] = mm_erc5.run_experiments(design=validation_data)

In [40]:
val['cm'].head(1)

,alpha,beta,input_flow,value_of_time,unit_cost_expansion,interest_rate,yield_curve,expand_capacity,amortization_period,debt_type,interest_rate_lock,no_build_travel_time,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion,present_cost_expansion
experiment,,,,,,,,,,,,,,,,,,
111,0.118865,4.595404,141,0.026021,127.842399,0.031256,-0.00218,41.407414,27,GO Bond,True,94.58819,67.037969,27.550221,101.082456,-190.952623,292.03508,5293.623191


In [41]:
val['erc'].head(1)

,alpha,beta,input_flow,value_of_time,unit_cost_expansion,interest_rate,yield_curve,expand_capacity,amortization_period,debt_type,interest_rate_lock,no_build_travel_time,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion,present_cost_expansion
experiment,,,,,,,,,,,,,,,,,,
111,0.118865,4.595404,141,0.026021,127.842399,0.031256,-0.00218,41.407414,27,GO Bond,True,96.821233,67.204716,28.015143,91.923001,-196.039541,274.859355,4910.682168


In [42]:
mm_erc.function.regression.predict(
    mm_erc.function.preprocess_raw_input(validation_data, to_type=float)
).head(1)

,no_build_travel_time,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion,present_cost_expansion
experiment,,,,,,,
111,3.632871,2.104709,3.332745,4.520951,-196.039541,5.61626,8.499168


In [43]:
mm_erc.function.regression._predict(
    mm_erc.function.preprocess_raw_input(validation_data, to_type=float)
)[:,0,:]

array([[   3.64246891,    3.63643421,    3.57238326,    3.62747379,
           3.67378754,    3.64467633],
       [   2.10823535,    2.12831627,    2.1069379 ,    2.14754804,
           2.03328881,    2.10392823],
       [   3.45845639,    3.2386993 ,    3.29467094,    3.37051493,
           3.24524631,    3.38888331],
       [   4.63915672,    4.57869837,    4.27123863,    4.5391098 ,
           4.39978514,    4.697719  ],
       [-191.73046569, -250.88232787, -151.0347015 , -196.64386412,
        -198.30039465, -187.64548976],
       [   5.77351365,    5.54709335,    5.60217215,    5.59837514,
           5.47869866,    5.69770422],
       [   8.61878256,    8.36570842,    8.54497382,    8.54029534,
           8.3751744 ,    8.55007432]])

In [44]:
mm_erc.function.regression.estimators_[1].predict(
    mm_erc.function.preprocess_raw_input(validation_data, to_type=float)
)[0]

array([   3.63643421,    2.12831627,    3.2386993 ,    4.57869837,
       -250.88232787,    5.54709335,    8.36570842])

In [45]:
mm_erc.function.regression.estimators_[1].estimators_[0].estimators_[0].coef_

array([ 5.00340699e+00,  4.01810944e-01,  5.45911896e-02,  3.37152951e+01,
       -1.59959354e-03,  4.16460123e+00,  4.42283662e+00,  1.72907294e-02,
        1.48089030e-03, -1.60679272e-01,  2.69196432e-02, -7.59860051e-03,
       -1.93210427e-02])

In [46]:
mm_erc.function.regression.estimators_[0].estimators_[0].estimators_[1].kernel_

RBF(length_scale=[0.157, 2.01, 216, 6.95e+04, 109, 0.022, 0.0895, 11.7, 126, 1e+05, 0.0209, 1.49e+03, 0.00156])

In [50]:

systems = ['cm','mm','rc','boo','sst','erc1','erc','erc5','lr']

# pandas.concat([
#     val[k].present_cost_expansion.rename(k)
#     for k in systems
# ], axis=1).head()

In [51]:
from sklearn.metrics import r2_score
scores = {}
for comp in systems:
    scores[comp] = pandas.Series({
        i: r2_score(val['cm'][i], val[comp][i])
        for i in s.get_measure_names()
    })
pandas.DataFrame.from_dict(scores).T

,no_build_travel_time,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion,present_cost_expansion
cm,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
mm,0.991611,0.900000,0.963781,0.878246,0.843721,0.918833,0.985319
rc,0.990185,0.854174,0.980190,0.955830,0.739517,0.965363,0.953689
boo,0.976984,0.929032,0.903375,0.884694,0.764306,0.920677,0.915053
sst,0.995840,0.971186,0.984708,0.902150,0.783985,0.991933,0.997227
erc1,0.993925,0.746095,0.969251,0.821054,0.809351,0.950306,0.946786
erc,0.984212,0.787676,0.943099,0.917562,0.861769,0.885932,0.882521
erc5,0.986858,0.794837,0.943983,0.903543,0.817583,0.884182,0.629309
lr,0.971534,0.624780,0.802494,0.663875,0.810166,0.508537,0.374351
